# Chapter 4 - Exercise 4: Inkjet Droplet Formation and Ejection Dynamics

## Learning Objectives
- Understand droplet formation mechanisms in inkjet bioprinting
- Apply dimensionless number analysis (Reynolds, Weber, Ohnesorge, Z-number)
- Predict jetting regimes and droplet behavior
- Calculate droplet velocity, trajectory, and impact dynamics
- Optimize inkjet parameters for stable bioprinting

## Background

**Inkjet bioprinting** ejects discrete droplets of cell-laden bioink with high precision. Success depends on achieving stable droplet formation without satellites or clogging.

### Actuation Methods (from Chapter 4.4.2):
- **Thermal inkjet**: Rapid heating creates vapor bubble → pressure pulse → droplet ejection
- **Piezoelectric inkjet**: Voltage pulse deforms piezo crystal → pressure wave → droplet ejection  
- **Electrostatic inkjet**: Electric field between nozzle and substrate pulls droplet
- **Electrohydrodynamic**: High voltage creates Taylor cone → fine droplet formation

### Key Dimensionless Numbers:

**Reynolds Number** (inertial vs viscous forces):
$$Re = \frac{\rho v d}{\eta}$$

**Weber Number** (inertial vs surface tension forces):
$$We = \frac{\rho v^2 d}{\sigma}$$

**Ohnesorge Number** (viscous vs surface tension forces):
$$Oh = \frac{\eta}{\sqrt{\rho \sigma d}} = \frac{\sqrt{We}}{Re}$$

**Z-number (Inverse Ohnesorge)** - Printability criterion:
$$Z = \frac{1}{Oh} = \frac{\sqrt{\rho \sigma d}}{\eta}$$

**Printable range: 1 < Z < 10**
- Z < 1: Too viscous (clogging)
- Z > 10: Too fluid (satellite droplets, poor control)

### Critical Parameters:
- Typical droplet diameter: **30-100 µm**
- Typical ejection velocity: **1-10 m/s**
- Typical frequency: **1,000-10,000 drops/s**
- Required viscosity: **1-10 mPa·s**

## Setup: Install and Import Libraries

In [ ]:
# Install required packages
import sys
!{sys.executable} -m pip install numpy matplotlib pandas seaborn scipy plotly -q

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.optimize import fsolve
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib.patches import Rectangle, FancyBboxPatch
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✓ All libraries loaded successfully!")
print("Ready to explore inkjet droplet dynamics!")

## Part 1: Dimensionless Number Calculator

In [ ]:
def calculate_dimensionless_numbers(density, viscosity, surface_tension, 
                                   velocity, diameter):
    """
    Calculate all relevant dimensionless numbers for inkjet printing.
    
    Parameters:
    -----------
    density : float
        Fluid density ρ (kg/m³)
    viscosity : float
        Dynamic viscosity η (Pa·s)
    surface_tension : float
        Surface tension σ (N/m)
    velocity : float
        Droplet velocity v (m/s)
    diameter : float
        Nozzle/droplet diameter d (m)
    
    Returns:
    --------
    dict : Dictionary containing all dimensionless numbers
    """
    # Reynolds number
    Re = (density * velocity * diameter) / viscosity
    
    # Weber number  
    We = (density * velocity**2 * diameter) / surface_tension
    
    # Ohnesorge number
    Oh = viscosity / np.sqrt(density * surface_tension * diameter)
    
    # Z-number (inverse Ohnesorge)
    Z = 1 / Oh
    
    # Capillary number (viscous vs surface tension)
    Ca = (viscosity * velocity) / surface_tension
    
    # Bond number (gravitational vs surface tension) - less relevant for inkjet
    g = 9.81  # m/s²
    Bo = (density * g * diameter**2) / surface_tension
    
    return {
        'Reynolds': Re,
        'Weber': We,
        'Ohnesorge': Oh,
        'Z_number': Z,
        'Capillary': Ca,
        'Bond': Bo
    }

def assess_printability(Z, Re, We):
    """
    Assess printability based on dimensionless numbers.
    
    Parameters:
    -----------
    Z : float
        Z-number (inverse Ohnesorge)
    Re : float
        Reynolds number
    We : float
        Weber number
    
    Returns:
    --------
    dict : Assessment with regime and recommendations
    """
    assessment = {
        'printable': False,
        'regime': '',
        'issues': [],
        'recommendations': []
    }
    
    # Z-number assessment (primary criterion)
    if 1 < Z < 10:
        assessment['printable'] = True
        assessment['regime'] = 'Stable jetting'
    elif Z <= 1:
        assessment['regime'] = 'Viscosity-dominated'
        assessment['issues'].append('Too viscous - risk of clogging')
        assessment['recommendations'].append('Reduce viscosity (dilute bioink or increase temperature)')
        assessment['recommendations'].append('Increase nozzle diameter')
    else:  # Z > 10
        assessment['regime'] = 'Inertia-dominated'
        assessment['issues'].append('Too fluid - satellite droplets and poor control')
        assessment['recommendations'].append('Increase viscosity (higher concentration)')
        assessment['recommendations'].append('Reduce nozzle diameter')
        assessment['recommendations'].append('Reduce ejection velocity')
    
    # Additional checks
    if Re < 1:
        assessment['issues'].append('Low Reynolds number - viscous forces dominate')
    elif Re > 100:
        assessment['issues'].append('High Reynolds number - turbulent effects possible')
    
    if We < 1:
        assessment['issues'].append('Low Weber number - surface tension prevents breakup')
    elif We > 100:
        assessment['issues'].append('High Weber number - excessive splashing on impact')
    
    return assessment

def calculate_droplet_properties(nozzle_diameter, ejection_velocity, 
                                viscosity, density, surface_tension):
    """
    Calculate droplet size, flight time, and impact characteristics.
    
    Parameters:
    -----------
    nozzle_diameter : float
        Nozzle diameter (µm)
    ejection_velocity : float
        Initial velocity (m/s)
    viscosity : float
        Dynamic viscosity (mPa·s)
    density : float
        Density (kg/m³)
    surface_tension : float
        Surface tension (mN/m)
    
    Returns:
    --------
    dict : Droplet properties
    """
    # Convert units
    d_nozzle_m = nozzle_diameter * 1e-6  # µm → m
    eta_Pa_s = viscosity * 1e-3  # mPa·s → Pa·s
    sigma_N_m = surface_tension * 1e-3  # mN/m → N/m
    
    # Droplet diameter (typically 1.5-2× nozzle diameter for inkjet)
    droplet_diameter_m = 1.8 * d_nozzle_m
    droplet_diameter_um = droplet_diameter_m * 1e6
    
    # Droplet volume
    droplet_volume_m3 = (4/3) * np.pi * (droplet_diameter_m/2)**3
    droplet_volume_pL = droplet_volume_m3 * 1e15  # m³ → pL
    
    # Print gap (typical: 0.5-2 mm)
    print_gap = 1e-3  # 1 mm
    
    # Flight time (neglecting air resistance and gravity for short distances)
    flight_time = print_gap / ejection_velocity
    
    # Impact velocity (approximately same as ejection for short distances)
    impact_velocity = ejection_velocity
    
    # Spreading diameter upon impact (Weber number dependent)
    We_impact = (density * impact_velocity**2 * droplet_diameter_m) / sigma_N_m
    # Empirical spreading correlation: D_max/D_0 ≈ 0.5 * We^0.25
    spreading_factor = 0.5 * We_impact**0.25
    spread_diameter_um = droplet_diameter_um * spreading_factor
    
    # Resolution (center-to-center spacing)
    resolution_um = spread_diameter_um
    
    return {
        'droplet_diameter_um': droplet_diameter_um,
        'droplet_volume_pL': droplet_volume_pL,
        'flight_time_ms': flight_time * 1000,
        'impact_velocity_ms': impact_velocity,
        'spread_diameter_um': spread_diameter_um,
        'resolution_um': resolution_um,
        'Weber_impact': We_impact
    }

print("✓ Dimensionless number calculation functions defined!")
print("\nKey functions:")
print("  • calculate_dimensionless_numbers()")
print("  • assess_printability()")
print("  • calculate_droplet_properties()")

## Part 2: Jetting Regime Map (Ohnesorge Diagram)

The classic diagram showing printability zones:

In [ ]:
# Create Ohnesorge diagram
fig, ax = plt.subplots(figsize=(12, 9))

# Create Reynolds vs Ohnesorge grid
Re_range = np.logspace(-1, 3, 300)
Oh_range = np.logspace(-2, 2, 300)

Re_grid, Oh_grid = np.meshgrid(Re_range, Oh_range)
We_grid = (Re_grid * Oh_grid)**2
Z_grid = 1 / Oh_grid

# Define jetting regimes
# Based on Fromm (1984) and Derby (2010)

# Regime 1: Stable jetting (1 < Z < 10, or 0.1 < Oh < 1)
stable_zone = (Z_grid >= 1) & (Z_grid <= 10)

# Regime 2: Viscosity-dominated (Z < 1, or Oh > 1)
viscous_zone = Z_grid < 1

# Regime 3: Satellite formation (Z > 10, or Oh < 0.1)  
satellite_zone = Z_grid > 10

# Plot regime zones
ax.contourf(Re_grid, Oh_grid, stable_zone.astype(float), 
           levels=[0.5, 1.5], colors=['lightgreen'], alpha=0.4)
ax.contourf(Re_grid, Oh_grid, viscous_zone.astype(float),
           levels=[0.5, 1.5], colors=['lightcoral'], alpha=0.4)
ax.contourf(Re_grid, Oh_grid, satellite_zone.astype(float),
           levels=[0.5, 1.5], colors=['lightyellow'], alpha=0.4)

# Add Z-number contour lines
Z_levels = [0.1, 1, 4, 10, 20, 50]
contours = ax.contour(Re_grid, Oh_grid, Z_grid, levels=Z_levels,
                     colors='black', linewidths=1.5, linestyles='dashed')
ax.clabel(contours, inline=True, fontsize=10, fmt='Z=%g')

# Mark printable window
Oh_printable = [0.1, 1]
ax.axhspan(Oh_printable[0], Oh_printable[1], alpha=0.15, color='green', 
          linewidth=3, edgecolor='darkgreen')

# Add example bioinks
bioinks_inkjet = {
    'Water': {'Re': 50, 'Oh': 0.02, 'color': 'blue'},
    'Dilute Alginate': {'Re': 20, 'Oh': 0.05, 'color': 'green'},
    'Collagen (low)': {'Re': 10, 'Oh': 0.15, 'color': 'orange'},
    'Gelatin (5%)': {'Re': 5, 'Oh': 0.3, 'color': 'red'},
    'Too viscous': {'Re': 1, 'Oh': 2, 'color': 'darkred'}
}

for bioink_name, props in bioinks_inkjet.items():
    ax.plot(props['Re'], props['Oh'], 'o', markersize=12, 
           color=props['color'], markeredgecolor='black', markeredgewidth=2,
           label=bioink_name)
    ax.annotate(bioink_name, xy=(props['Re'], props['Oh']),
               xytext=(10, 10), textcoords='offset points',
               fontsize=9, fontweight='bold',
               bbox=dict(boxstyle='round,pad=0.3', facecolor=props['color'], 
                        alpha=0.5, edgecolor='black'))

# Labels and formatting
ax.set_xlabel('Reynolds Number, Re = ρvd/η', fontsize=13, fontweight='bold')
ax.set_ylabel('Ohnesorge Number, Oh = η/√(ρσd)', fontsize=13, fontweight='bold')
ax.set_title('Inkjet Jetting Regime Map (Ohnesorge Diagram)\nPrintable Window: 1 < Z < 10 (0.1 < Oh < 1)', 
            fontsize=14, fontweight='bold', pad=20)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(0.1, 1000)
ax.set_ylim(0.01, 10)
ax.grid(True, alpha=0.3, which='both')

# Add regime labels
ax.text(50, 0.03, 'SATELLITE\nDROPLETS\n(Z > 10)', 
       ha='center', va='center', fontsize=11, fontweight='bold',
       bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.6))
ax.text(50, 0.3, 'STABLE\nJETTING\n(1 < Z < 10)', 
       ha='center', va='center', fontsize=12, fontweight='bold',
       bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7, 
                edgecolor='darkgreen', linewidth=2))
ax.text(1, 3, 'NO JETTING\nCLOGGING\n(Z < 1)', 
       ha='center', va='center', fontsize=11, fontweight='bold',
       bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.6))

plt.tight_layout()
plt.show()

print("\n💡 Key Insights from Ohnesorge Diagram:")
print("   • GREEN ZONE (0.1 < Oh < 1): Stable droplet formation")
print("     → Optimal for bioprinting")
print("   • YELLOW ZONE (Oh < 0.1): Satellite droplets")
print("     → Too fluid, poor control")
print("   • RED ZONE (Oh > 1): Viscosity-dominated")
print("     → Too viscous, nozzle clogging")
print("\n   • Most bioinks must be diluted to reach printable window")
print("   • Water and dilute alginate are in stable zone")
print("   • Concentrated proteins (gelatin, collagen) are too viscous")

## Part 3: Interactive Inkjet Parameter Calculator

### 🎯 STUDENT TASK 1: Optimize Inkjet Parameters for Your Bioink

**Modify the parameters below to achieve stable jetting (1 < Z < 10):**

In [ ]:
# ==========================================
# STUDENT PARAMETERS - MODIFY THESE VALUES
# ==========================================

# Bioink properties
bioink_viscosity = 5          # mPa·s (try: 1, 5, 10, 20)
bioink_density = 1000         # kg/m³ (typically 1000-1050 for aqueous bioinks)
bioink_surface_tension = 50   # mN/m (try: 30, 50, 70)
                              # Water ~72, bioinks typically 40-60

# Inkjet parameters
nozzle_diameter = 50          # µm (try: 30, 50, 80, 100)
ejection_velocity = 5         # m/s (try: 1, 3, 5, 8, 10)

# ==========================================

# Convert units for calculations
eta_Pa_s = bioink_viscosity * 1e-3
sigma_N_m = bioink_surface_tension * 1e-3
d_m = nozzle_diameter * 1e-6

# Calculate dimensionless numbers
numbers = calculate_dimensionless_numbers(
    bioink_density, eta_Pa_s, sigma_N_m, ejection_velocity, d_m
)

# Assess printability
assessment = assess_printability(numbers['Z_number'], numbers['Reynolds'], numbers['Weber'])

# Calculate droplet properties
droplet_props = calculate_droplet_properties(
    nozzle_diameter, ejection_velocity, bioink_viscosity, 
    bioink_density, bioink_surface_tension
)

# Create comprehensive visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Dimensionless Numbers', 'Z-Number Assessment',
                   'Droplet Formation Sequence', 'Performance Metrics'),
    specs=[[{'type': 'bar'}, {'type': 'indicator'}],
           [{'type': 'scatter'}, {'type': 'table'}]]
)

# Plot 1: Dimensionless numbers bar chart
dim_names = ['Reynolds', 'Weber', 'Ohnesorge', 'Z-number']
dim_values = [numbers['Reynolds'], numbers['Weber'], 
             numbers['Ohnesorge'], numbers['Z_number']]
colors_dim = ['#3498db', '#2ecc71', '#f39c12', '#e74c3c']

fig.add_trace(
    go.Bar(x=dim_names, y=dim_values, marker_color=colors_dim,
          text=[f'{v:.2f}' for v in dim_values], textposition='outside'),
    row=1, col=1
)

fig.update_yaxes(title_text="Value", type="log", row=1, col=1)

# Plot 2: Z-number gauge
Z = numbers['Z_number']

if 1 < Z < 10:
    gauge_color = "green"
    status = "PRINTABLE"
elif Z <= 1:
    gauge_color = "red"
    status = "TOO VISCOUS"
else:
    gauge_color = "orange"
    status = "TOO FLUID"

fig.add_trace(
    go.Indicator(
        mode="gauge+number+delta",
        value=Z,
        title={'text': f"Z-Number<br>{status}"},
        delta={'reference': 4, 'valueformat': '.2f'},
        gauge={
            'axis': {'range': [None, 20], 'tickmode': 'linear', 'tick0': 0, 'dtick': 5},
            'bar': {'color': gauge_color},
            'steps': [
                {'range': [0, 1], 'color': "lightgray"},
                {'range': [1, 10], 'color': "lightgreen"},
                {'range': [10, 20], 'color': "lightyellow"}],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': 10}
        }),
    row=1, col=2
)

# Plot 3: Droplet formation sequence (schematic)
time_sequence = np.linspace(0, 1, 6)
# Simplified droplet shape evolution
for i, t in enumerate(time_sequence):
    # Droplet position
    y_pos = 0.5 - t * 0.4
    
    # Draw nozzle
    if i == 0:
        fig.add_shape(
            type="rect",
            x0=-0.05, x1=0.05, y0=0.5, y1=0.6,
            fillcolor="gray", line=dict(color="black", width=2),
            row=2, col=1
        )
    
    # Draw droplet
    radius = 0.03 * (1 + 0.5*np.sin(t*np.pi))  # Oscillating radius
    
    theta = np.linspace(0, 2*np.pi, 50)
    x_circle = t*0.2 + radius * np.cos(theta)
    y_circle = y_pos + radius * np.sin(theta) * 0.7  # Elongated
    
    fig.add_trace(
        go.Scatter(x=x_circle, y=y_circle, fill='toself',
                  fillcolor='lightblue', line=dict(color='blue', width=2),
                  showlegend=False, hoverinfo='skip'),
        row=2, col=1
    )

fig.update_xaxes(showticklabels=False, showgrid=False, zeroline=False, row=2, col=1)
fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False, row=2, col=1)

# Plot 4: Performance metrics table
fig.add_trace(
    go.Table(
        header=dict(values=['Parameter', 'Value'],
                   fill_color='paleturquoise',
                   align='left',
                   font=dict(size=12, color='black')),
        cells=dict(values=[
            ['Droplet Diameter', 'Droplet Volume', 'Flight Time', 
             'Impact Velocity', 'Spread Diameter', 'Resolution'],
            [f"{droplet_props['droplet_diameter_um']:.1f} µm",
             f"{droplet_props['droplet_volume_pL']:.2f} pL",
             f"{droplet_props['flight_time_ms']:.2f} ms",
             f"{droplet_props['impact_velocity_ms']:.1f} m/s",
             f"{droplet_props['spread_diameter_um']:.1f} µm",
             f"{droplet_props['resolution_um']:.1f} µm"]
        ],
        fill_color='lavender',
        align='left',
        font=dict(size=11))),
    row=2, col=2
)

fig.update_layout(height=900, showlegend=False,
                 title_text=f"Inkjet Bioprinting Analysis - {bioink_viscosity} mPa·s Bioink")
fig.show()

# Detailed text output
print("="*70)
print("INKJET BIOPRINTING ANALYSIS")
print("="*70)
print(f"\nBioink Properties:")
print(f"  • Viscosity: {bioink_viscosity} mPa·s")
print(f"  • Density: {bioink_density} kg/m³")
print(f"  • Surface tension: {bioink_surface_tension} mN/m")

print(f"\nInkjet Parameters:")
print(f"  • Nozzle diameter: {nozzle_diameter} µm")
print(f"  • Ejection velocity: {ejection_velocity} m/s")

print(f"\nDimensionless Numbers:")
print(f"  • Reynolds number: {numbers['Reynolds']:.2f}")
print(f"  • Weber number: {numbers['Weber']:.2f}")
print(f"  • Ohnesorge number: {numbers['Ohnesorge']:.3f}")
print(f"  • Z-number: {numbers['Z_number']:.2f}")
print(f"  • Capillary number: {numbers['Capillary']:.4f}")

print(f"\nPrintability Assessment:")
print(f"  • Status: {'✅ PRINTABLE' if assessment['printable'] else '❌ NOT PRINTABLE'}")
print(f"  • Regime: {assessment['regime']}")

if assessment['issues']:
    print(f"\n  Issues:")
    for issue in assessment['issues']:
        print(f"    ⚠️  {issue}")

if assessment['recommendations']:
    print(f"\n  Recommendations:")
    for rec in assessment['recommendations']:
        print(f"    💡 {rec}")

print(f"\nDroplet Properties:")
print(f"  • Droplet diameter: {droplet_props['droplet_diameter_um']:.1f} µm")
print(f"  • Droplet volume: {droplet_props['droplet_volume_pL']:.2f} pL")
print(f"  • Flight time: {droplet_props['flight_time_ms']:.2f} ms")
print(f"  • Impact Weber number: {droplet_props['Weber_impact']:.2f}")
print(f"  • Spread diameter: {droplet_props['spread_diameter_um']:.1f} µm")
print(f"  • Achievable resolution: {droplet_props['resolution_um']:.1f} µm")

# Optimization suggestions
if not assessment['printable']:
    print(f"\n💡 Optimization Strategy:")
    if numbers['Z_number'] < 1:
        # Too viscous
        target_Z = 4
        required_reduction = numbers['Z_number'] / target_Z
        
        new_viscosity = bioink_viscosity * required_reduction
        new_diameter = nozzle_diameter / (required_reduction**0.5)
        
        print(f"  To achieve Z ≈ {target_Z}:")
        print(f"    • Reduce viscosity to {new_viscosity:.1f} mPa·s (dilute bioink)")
        print(f"    OR increase nozzle to {new_diameter:.0f} µm")
        print(f"    OR increase temperature (reduces viscosity)")
    else:
        # Too fluid
        target_Z = 6
        required_increase = target_Z / numbers['Z_number']
        
        new_viscosity = bioink_viscosity * required_increase
        new_diameter = nozzle_diameter * (required_increase**0.5)
        
        print(f"  To achieve Z ≈ {target_Z}:")
        print(f"    • Increase viscosity to {new_viscosity:.1f} mPa·s (higher concentration)")
        print(f"    OR reduce nozzle to {new_diameter:.0f} µm")
        print(f"    OR reduce ejection velocity")

print("\n" + "="*70)

## Part 4: Z-Number Design Space

Explore how viscosity and nozzle diameter affect printability:

In [ ]:
# Create parameter sweep
viscosities = np.linspace(0.5, 15, 100)  # mPa·s
nozzle_diameters = np.linspace(20, 100, 100)  # µm

V_grid, D_grid = np.meshgrid(viscosities, nozzle_diameters)

# Fixed parameters
density_fixed = 1000  # kg/m³
surface_tension_fixed = 50  # mN/m

# Calculate Z-number for each combination
Z_grid = np.zeros_like(V_grid)

for i in range(V_grid.shape[0]):
    for j in range(V_grid.shape[1]):
        eta_Pa_s = V_grid[i, j] * 1e-3
        d_m = D_grid[i, j] * 1e-6
        sigma_N_m = surface_tension_fixed * 1e-3
        
        # Z = √(ρ·σ·d) / η
        Z_grid[i, j] = np.sqrt(density_fixed * sigma_N_m * d_m) / eta_Pa_s

# Create visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 14))

# Plot 1: Z-number contour map
contour1 = ax1.contourf(V_grid, D_grid, Z_grid, levels=20, cmap='RdYlGn')
contour1_lines = ax1.contour(V_grid, D_grid, Z_grid, levels=[1, 4, 10],
                             colors=['red', 'green', 'orange'], linewidths=3)
ax1.clabel(contour1_lines, inline=True, fontsize=12, fmt='Z=%g')
cbar1 = plt.colorbar(contour1, ax=ax1)
cbar1.set_label('Z-Number', fontsize=12, fontweight='bold')

# Highlight printable zone
printable_zone = (Z_grid >= 1) & (Z_grid <= 10)
ax1.contour(V_grid, D_grid, printable_zone.astype(float), levels=[0.5],
           colors=['darkgreen'], linewidths=4, linestyles='solid')

ax1.set_xlabel('Viscosity (mPa·s)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Nozzle Diameter (µm)', fontsize=12, fontweight='bold')
ax1.set_title('Z-Number Design Space\n(ρ=1000 kg/m³, σ=50 mN/m)', 
             fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Plot 2: Printable zone overlay
ax2.contourf(V_grid, D_grid, printable_zone.astype(float),
            levels=[0.5, 1.5], colors=['white', 'lightgreen'], alpha=0.7)
ax2.contour(V_grid, D_grid, Z_grid, levels=[1, 10],
           colors=['darkgreen'], linewidths=3, linestyles='solid')

# Mark example bioinks
examples = [
    {'name': 'Water', 'visc': 1, 'diam': 50, 'color': 'blue'},
    {'name': 'Dilute\nAlginate', 'visc': 3, 'diam': 60, 'color': 'green'},
    {'name': 'Collagen', 'visc': 8, 'diam': 80, 'color': 'orange'},
    {'name': 'Gelatin\n(too high)', 'visc': 12, 'diam': 50, 'color': 'red'}
]

for ex in examples:
    ax2.plot(ex['visc'], ex['diam'], 'o', markersize=15,
            color=ex['color'], markeredgecolor='black', markeredgewidth=2)
    ax2.annotate(ex['name'], xy=(ex['visc'], ex['diam']),
                xytext=(10, 10), textcoords='offset points',
                fontsize=9, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor=ex['color'], 
                         alpha=0.6, edgecolor='black'))

ax2.set_xlabel('Viscosity (mPa·s)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Nozzle Diameter (µm)', fontsize=12, fontweight='bold')
ax2.set_title('Printable Zone (1 < Z < 10)\nwith Example Bioinks', 
             fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Plot 3: Effect of surface tension
viscosity_test = 5  # mPa·s
diameter_range = np.linspace(20, 100, 100)
surface_tensions = [30, 50, 70]  # mN/m
colors_st = ['blue', 'green', 'red']

for sigma, color in zip(surface_tensions, colors_st):
    Z_values = []
    for d in diameter_range:
        eta_Pa_s = viscosity_test * 1e-3
        d_m = d * 1e-6
        sigma_N_m = sigma * 1e-3
        Z = np.sqrt(density_fixed * sigma_N_m * d_m) / eta_Pa_s
        Z_values.append(Z)
    
    ax3.plot(diameter_range, Z_values, linewidth=3, color=color,
            label=f'σ = {sigma} mN/m')

# Add printable zone
ax3.axhspan(1, 10, alpha=0.2, color='green', label='Printable zone')
ax3.axhline(1, color='red', linestyle='--', linewidth=2)
ax3.axhline(10, color='orange', linestyle='--', linewidth=2)

ax3.set_xlabel('Nozzle Diameter (µm)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Z-Number', fontsize=12, fontweight='bold')
ax3.set_title(f'Effect of Surface Tension\n(η = {viscosity_test} mPa·s fixed)',
             fontsize=13, fontweight='bold')
ax3.legend(loc='best', frameon=True, fancybox=True)
ax3.grid(True, alpha=0.3)
ax3.set_ylim(0, 15)

# Plot 4: Resolution vs viscosity trade-off
# For different nozzle sizes, show achievable resolution
nozzle_sizes = [30, 50, 80, 100]  # µm
visc_range = np.linspace(1, 10, 50)
colors_nozzle = ['purple', 'blue', 'green', 'orange']

for nozzle, color in zip(nozzle_sizes, colors_nozzle):
    printable_visc = []
    for v in visc_range:
        eta_Pa_s = v * 1e-3
        d_m = nozzle * 1e-6
        sigma_N_m = 50 * 1e-3
        Z = np.sqrt(density_fixed * sigma_N_m * d_m) / eta_Pa_s
        
        if 1 < Z < 10:
            printable_visc.append(v)
    
    if printable_visc:
        # Resolution approximately equals droplet spread (1.8× nozzle diameter)
        resolution = nozzle * 1.8
        ax4.barh(f'{nozzle} µm nozzle', len(printable_visc)/len(visc_range)*100,
                color=color, alpha=0.7, edgecolor='black', linewidth=2)
        ax4.text(len(printable_visc)/len(visc_range)*100 + 2, 
                nozzle_sizes.index(nozzle),
                f'Resolution: {resolution:.0f} µm',
                va='center', fontweight='bold')

ax4.set_xlabel('Printable Viscosity Range (%)', fontsize=12, fontweight='bold')
ax4.set_title('Nozzle Size vs Printable Range\nand Resolution',
             fontsize=13, fontweight='bold')
ax4.set_xlim(0, 110)
ax4.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n💡 Design Space Analysis:")
print("   • Larger nozzles → wider printable viscosity range")
print("   • Smaller nozzles → better resolution BUT narrower window")
print("   • Higher surface tension → easier to print (higher Z)")
print("   • Most bioinks need viscosity < 10 mPa·s for inkjet")
print("   • Optimal: 50-60 µm nozzle with 3-7 mPa·s bioink")

## Part 5: Droplet Trajectory and Impact Simulation

### 🎯 STUDENT TASK 2: Analyze Droplet Flight Dynamics

**Model droplet trajectory including air resistance:**

In [ ]:
# ==========================================
# STUDENT PARAMETERS - MODIFY THESE VALUES
# ==========================================

# Droplet properties
droplet_diameter_sim = 80    # µm (try: 50, 80, 100)
initial_velocity_sim = 5     # m/s (try: 2, 5, 8)
print_gap_sim = 1.0          # mm (try: 0.5, 1.0, 2.0)

# Bioink properties
density_sim = 1000           # kg/m³
viscosity_sim = 5            # mPa·s

# ==========================================

def simulate_droplet_trajectory(d_droplet_um, v0_ms, gap_mm, rho_kgm3, eta_mPas):
    """
    Simulate droplet trajectory with air drag.
    """
    # Convert units
    d_m = d_droplet_um * 1e-6
    gap_m = gap_mm * 1e-3
    
    # Air properties
    rho_air = 1.2  # kg/m³
    eta_air = 1.8e-5  # Pa·s
    
    # Droplet mass
    volume = (4/3) * np.pi * (d_m/2)**3
    mass = rho_kgm3 * volume
    
    # Reynolds number for droplet in air
    Re_air = (rho_air * v0_ms * d_m) / eta_air
    
    # Drag coefficient (Stokes for Re < 1, empirical for higher Re)
    if Re_air < 1:
        Cd = 24 / Re_air
    else:
        Cd = 24/Re_air * (1 + 0.15*Re_air**0.687)
    
    # Simulation
    g = 9.81  # m/s²
    dt = 1e-6  # s (1 µs time step)
    t_max = gap_m / (v0_ms * 0.5)  # Estimated max time
    
    t = 0
    y = 0  # Vertical position
    v = v0_ms  # Velocity
    
    trajectory_t = [t]
    trajectory_y = [y]
    trajectory_v = [v]
    
    while y < gap_m and t < t_max:
        # Update Reynolds and drag
        Re_air = (rho_air * v * d_m) / eta_air
        if Re_air < 1:
            Cd = 24 / max(Re_air, 0.1)
        else:
            Cd = 24/Re_air * (1 + 0.15*Re_air**0.687)
        
        # Drag force
        A_cross = np.pi * (d_m/2)**2
        F_drag = 0.5 * Cd * rho_air * v**2 * A_cross
        
        # Net force (gravity + drag, both downward after ejection)
        # Assuming vertical downward printing
        F_net = mass * g - F_drag  # Drag opposes motion
        
        # Acceleration
        a = F_net / mass
        
        # Update velocity and position
        v = v + a * dt
        y = y + v * dt
        t = t + dt
        
        # Store every 100 steps to reduce array size
        if len(trajectory_t) % 100 == 0:
            trajectory_t.append(t)
            trajectory_y.append(y)
            trajectory_v.append(v)
    
    return {
        'time': np.array(trajectory_t),
        'position': np.array(trajectory_y) * 1000,  # Convert to mm
        'velocity': np.array(trajectory_v),
        'impact_velocity': v,
        'flight_time': t * 1000  # Convert to ms
    }

# Run simulation
trajectory = simulate_droplet_trajectory(
    droplet_diameter_sim, initial_velocity_sim, print_gap_sim,
    density_sim, viscosity_sim
)

# Create visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Position vs time
ax1.plot(trajectory['time'] * 1000, trajectory['position'], 
        linewidth=3, color='#3498db')
ax1.axhline(print_gap_sim, color='red', linestyle='--', linewidth=2,
           label=f'Substrate ({print_gap_sim} mm)')
ax1.set_xlabel('Time (ms)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Position (mm)', fontsize=12, fontweight='bold')
ax1.set_title('Droplet Trajectory', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Velocity vs time
ax2.plot(trajectory['time'] * 1000, trajectory['velocity'],
        linewidth=3, color='#2ecc71')
ax2.axhline(initial_velocity_sim, color='blue', linestyle=':', 
           linewidth=2, label='Initial velocity')
ax2.axhline(trajectory['impact_velocity'], color='red', linestyle='--',
           linewidth=2, label='Impact velocity')
ax2.set_xlabel('Time (ms)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Velocity (m/s)', fontsize=12, fontweight='bold')
ax2.set_title('Droplet Velocity', fontsize=13, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Schematic of impact
ax3.axis('equal')
ax3.set_xlim(-2, 2)
ax3.set_ylim(-0.5, 2)

# Draw substrate
ax3.add_patch(Rectangle((-2, -0.3), 4, 0.3, facecolor='gray', 
                        edgecolor='black', linewidth=2))
ax3.text(0, -0.15, 'Substrate', ha='center', va='center',
        fontsize=12, fontweight='bold', color='white')

# Draw droplet at impact
circle = plt.Circle((0, 0.1), 0.3, color='lightblue', 
                   edgecolor='blue', linewidth=2)
ax3.add_patch(circle)

# Draw velocity arrow
ax3.arrow(0, 0.8, 0, -0.4, head_width=0.2, head_length=0.1,
         fc='red', ec='darkred', linewidth=3)
ax3.text(0.5, 0.6, f'v = {trajectory["impact_velocity"]:.2f} m/s',
        fontsize=11, fontweight='bold')

# Draw spread
We_impact = (density_sim * trajectory['impact_velocity']**2 * 
            droplet_diameter_sim*1e-6) / (50e-3)
spread_factor = 0.5 * We_impact**0.25
spread_radius = (droplet_diameter_sim * 1e-6 * spread_factor / 2) * 1000  # Convert to mm

ellipse = plt.matplotlib.patches.Ellipse((0, 0), spread_radius*2, 0.2,
                                        facecolor='lightblue', 
                                        edgecolor='blue',
                                        linewidth=2, alpha=0.5)
ax3.add_patch(ellipse)
ax3.text(0, -0.6, f'Spread: {spread_radius*2:.0f} mm',
        ha='center', fontsize=10, fontweight='bold')

ax3.set_title('Droplet Impact Schematic', fontsize=13, fontweight='bold')
ax3.axis('off')

# Plot 4: Summary table
ax4.axis('tight')
ax4.axis('off')

summary_data = [
    ['Parameter', 'Value'],
    ['Initial Velocity', f'{initial_velocity_sim:.1f} m/s'],
    ['Impact Velocity', f'{trajectory["impact_velocity"]:.2f} m/s'],
    ['Velocity Loss', f'{(1 - trajectory["impact_velocity"]/initial_velocity_sim)*100:.1f}%'],
    ['Flight Time', f'{trajectory["flight_time"]:.2f} ms'],
    ['Print Gap', f'{print_gap_sim:.1f} mm'],
    ['Droplet Diameter', f'{droplet_diameter_sim} µm'],
    ['Impact Weber #', f'{We_impact:.2f}'],
    ['Spread Diameter', f'{spread_radius*2*1000:.0f} µm']
]

table = ax4.table(cellText=summary_data, cellLoc='left', loc='center',
                 colWidths=[0.5, 0.5])
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 2.5)

# Color header
for i in range(2):
    table[(0, i)].set_facecolor('#34495e')
    table[(0, i)].set_text_props(weight='bold', color='white')

ax4.set_title('Flight & Impact Summary', fontsize=13, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

# Text output
print("="*70)
print("DROPLET TRAJECTORY SIMULATION")
print("="*70)
print(f"\nSimulation Parameters:")
print(f"  • Droplet diameter: {droplet_diameter_sim} µm")
print(f"  • Initial velocity: {initial_velocity_sim} m/s")
print(f"  • Print gap: {print_gap_sim} mm")

print(f"\nResults:")
print(f"  • Flight time: {trajectory['flight_time']:.2f} ms")
print(f"  • Impact velocity: {trajectory['impact_velocity']:.2f} m/s")
print(f"  • Velocity loss: {(1-trajectory['impact_velocity']/initial_velocity_sim)*100:.1f}%")
print(f"  • Impact Weber number: {We_impact:.2f}")
print(f"  • Spread diameter: {spread_radius*2*1000:.0f} µm")

print(f"\nInterpretation:")
if trajectory['flight_time'] < 0.5:
    print("  ✓ Fast flight - minimal evaporation concern")
else:
    print("  ⚠️  Long flight - evaporation may affect droplet size")

if We_impact < 10:
    print("  ✓ Low Weber number - gentle impact, minimal splashing")
elif We_impact < 50:
    print("  ✓ Moderate Weber number - good spreading")
else:
    print("  ⚠️  High Weber number - risk of splashing")

if spread_radius*2*1000 < 100:
    print(f"  ✓ Resolution: {spread_radius*2*1000:.0f} µm - excellent for cell patterning")
else:
    print(f"  ⚠️  Resolution: {spread_radius*2*1000:.0f} µm - consider smaller droplets")

print("\n" + "="*70)

## Part 6: Summary and Key Takeaways

In [ ]:
print("="*80)
print("CHAPTER 4 EXERCISE 4: KEY LEARNING POINTS")
print("="*80)
print("""
1. Z-NUMBER IS THE PRIMARY PRINTABILITY CRITERION
   → Z = √(ρ·σ·d) / η  (inverse Ohnesorge number)
   → Printable range: 1 < Z < 10
   → Z < 1: Too viscous (clogging)
   → Z > 10: Too fluid (satellites, poor control)
   → Z ≈ 4: Optimal for most applications

2. DIMENSIONLESS NUMBERS REVEAL PHYSICS
   → Reynolds (Re): Inertial vs viscous forces
   → Weber (We): Inertial vs surface tension forces
   → Ohnesorge (Oh): Viscous vs surface tension forces
   → These numbers are scale-independent and universal!

3. VISCOSITY CONSTRAINTS ARE SEVERE
   → Inkjet requires: η = 1-10 mPa·s
   → Most native bioinks are too viscous
   → Common solutions:
     • Dilution (reduces viscosity but also cell density)
     • Temperature control (heating reduces viscosity)
     • Surfactants (reduce surface tension)

4. NOZZLE DIAMETER TRADE-OFFS
   → Smaller nozzles (30-50 µm):
     • Better resolution
     • Narrower printable viscosity range
     • Higher clogging risk
   → Larger nozzles (60-100 µm):
     • Wider viscosity tolerance
     • Lower resolution
     • More robust operation

5. SURFACE TENSION IS YOUR FRIEND
   → Higher σ → Higher Z → Easier printing
   → BUT surfactants (reduce σ) sometimes needed to prevent clogging
   → Trade-off: printability vs droplet control

6. DROPLET FORMATION MECHANISMS MATTER
   → Thermal: Simple, but heat may damage cells
   → Piezoelectric: Gentle, precise, more complex
   → Electrohydrodynamic: Finest droplets, requires conductive bioink
   → All require Z in printable range!

7. IMPACT DYNAMICS AFFECT RESOLUTION
   → Droplet spreads upon impact (We-dependent)
   → Spread diameter ≈ 1.5-2.5× initial droplet diameter
   → Resolution = spread diameter (~50-150 µm typical)
   → Lower impact velocity → less spreading → better resolution

8. PRACTICAL DESIGN GUIDELINES
   → Target: Z = 3-6 for robust operation
   → Nozzle: 50-70 µm for most applications
   → Viscosity: 3-7 mPa·s (requires dilution for most bioinks)
   → Velocity: 3-8 m/s (balance between speed and control)
   → Frequency: 1-10 kHz for high throughput
   → Print gap: 0.5-2 mm (closer = better accuracy)

9. CELL CONSIDERATIONS
   → Shear stress in nozzle typically < 150 Pa (safe)
   → Impact forces minimal due to soft substrate/bath
   → Main concern: Thermal damage (thermal inkjet)
   → Cell density limited: < 10⁷ cells/mL to prevent clogging

10. COMPLEMENTARY TO EXTRUSION
    → Inkjet: High resolution, low viscosity, fast
    → Extrusion: Large constructs, high viscosity, slow
    → Hybrid systems combine both!
""")
print("="*80)

## 🎓 REFLECTION QUESTIONS

### Question 1
**Derive the Z-number equation starting from the Ohnesorge number. Explain physically what each term represents and why Z (not Oh) is used as the printability criterion.**

### Question 2  
**Your bioink has η = 12 mPa·s, ρ = 1000 kg/m³, σ = 50 mN/m. You want to print with a 60 µm nozzle. Calculate Z and determine if it's printable. If not, what changes would make it printable?**

### Question 3
**Explain why satellite droplets form when Z > 10. Use the concepts of inertia, viscosity, and surface tension in your explanation.**

### Question 4
**Compare the advantages and disadvantages of thermal vs piezoelectric inkjet for bioprinting hepatocytes (temperature-sensitive cells). Which would you choose and why?**

### Question 5
**A researcher achieves Z = 5 with a 50 µm nozzle but wants to improve resolution by using a 30 µm nozzle. What happens to Z? Will the system still be printable? What adjustments might be needed?**

## 📚 Additional Challenges (Optional)

### Challenge 1: Evaporation Effects
Model water evaporation during flight. For a 70 µm droplet at 25°C with 50% humidity, how much volume is lost during 1 ms flight? How does this affect Z?

### Challenge 2: Multi-Nozzle Systems
Design a multi-nozzle printhead with 8 nozzles. Consider: nozzle spacing, crosstalk, synchronized jetting. What throughput can you achieve?

### Challenge 3: Coaxial Inkjet
Research coaxial inkjet systems (core-shell droplets). Calculate Z-number for both inner and outer fluids. When is this advantageous?

### Challenge 4: Experimental Design
Design an experiment to measure actual droplet velocity and diameter. What imaging technique would you use? What frame rate is needed?

## 🎯 Congratulations!

You've completed Exercise 4: Inkjet Droplet Formation and Ejection Dynamics!

**You now understand:**
- ✓ Dimensionless number analysis (Re, We, Oh, Z)
- ✓ The Z-number criterion for printability (1 < Z < 10)
- ✓ Jetting regime maps and Ohnesorge diagrams
- ✓ Viscosity constraints and bioink formulation strategies
- ✓ Droplet trajectory, impact, and spreading dynamics
- ✓ Trade-offs between resolution, robustness, and throughput

**Next Steps:**
- Continue to Exercise 5: Photopolymerization Kinetics (Light-based Bioprinting)
- Review Chapter 4.4.2 for deeper understanding of inkjet mechanisms
- Explore research papers on droplet-based bioprinting applications

---

*Exercise created for Biofabrication Chapter 4 - Master's Level Bioengineering*